In [0]:
from pyspark.sql.functions import count, countDistinct, sum, rank, desc, round, avg, concat, lpad, col, lit, dayofweek, when
from pyspark.sql import Window

from pyspark.sql.types import StringType 

In [0]:
dbutils.widgets.text("csv_url", "url")
dbutils.widgets.text("par_url", "url")

csv_file = dbutils.widgets.get("csv_url")
parquet_file = dbutils.widgets.get("par_url")

df = spark.read.table("merchants_transaction_clean")

In [0]:
table_exists = spark.catalog.tableExists("merchants_transaction_clean")

if not table_exists:
    print("Table 'merchants_transaction_clean' does not exist")
    print("Running preparation notebook in order to create it")
    result = dbutils.notebook.run('preparation', 0, {"csv_url": csv_file, "par_url": parquet_file})
    print(result)
else:
    print("Table 'merchants_transaction_clean' exists and is ready to be used for analysis")

### question 1: 
Generate the top 5 merchants by purchase_amount for each month in the dataset for each city in the dataset.

In [0]:
dfq1_prep = df.where("authorized_flag = 'Y' AND calc_city_id != -1 AND merchant_name != 'Unknown'")\
    .selectExpr("purchase_year_month", "calc_city_id AS city_id", "merchant_name", "purchase_amount")\
    .groupBy("purchase_year_month", "city_id", "merchant_name")\
    .agg(sum("purchase_amount").alias("purchase_total"), count("*").alias("no_of_sales"))

In [0]:
dfq1_ranked = dfq1_prep.withColumn("sales_rank", rank().over(Window.partitionBy("purchase_year_month", "city_id").orderBy(desc("purchase_total"))))

In [0]:
dfq1_final = dfq1_ranked.withColumn("purchase_total", round("purchase_total",0)).where("sales_rank <= 5").drop("sales_rank")

In [0]:
display(dfq1_final.limit(50))

### question 2: 
What is the average sale amount (purchase_amount) of each merchant in each state. 
Consider returning the merchants with the largest sales first:

In [0]:
dfq2_final = df.where("authorized_flag = 'Y' AND calc_state_id != -1 AND merchant_name != 'Unknown'")\
    .selectExpr("merchant_name", "calc_state_id AS state_id", "purchase_amount")\
    .groupBy("merchant_name", "state_id")\
    .agg(avg("purchase_amount").alias("avg_sales"))\
    .withColumn("avg_sales", round("avg_sales", 0))\
    .orderBy(desc("avg_sales"))

In [0]:
display(dfq2_final.limit(10))

### question 3: 
Identify the top 3 hours where the largest amount of sales (purchase_amount) are
recorded for each product category (category).

In [0]:
dfq3_prep = df.where("authorized_flag = 'Y' AND category != 'Unknown'")\
    .select("category", "purchase_hour")\
    .withColumn("purchase_hour", concat(lpad(col("purchase_hour").cast(StringType()),2,0), lit("00")))\
    .groupBy("category", "purchase_hour").agg(count("*").alias("sales_amount")).orderBy(desc("sales_amount"))

In [0]:
dfq3_ranked = dfq3_prep.withColumn("sales_rank", rank().over(Window.partitionBy("category").orderBy(desc("sales_amount"))))

In [0]:
dfq3_final = dfq3_ranked.withColumn("sales_amount", round("sales_amount",0)).where("sales_rank <= 3").drop("sales_rank").drop("sales_amount")

In [0]:
display(dfq3_final.limit(100))

### question 4: 
In which cities are the most popular merchants located. Is there a correlation
between the location (city_id) and the categories (category) the merchant sells.

In [0]:
dfq4_prep = df.where("authorized_flag = 'Y' AND merchant_id IS NOT NULL")\
    .select("merchant_id", "purchase_amount", "purchase_year_month")\
    .groupBy("merchant_id")\
    .agg(count("*").alias("total_transactions"), sum("purchase_amount").alias("total_revenue"), countDistinct("purchase_year_month").alias("active_months"))\
    .withColumn("avg_tx_per_month", col("total_transactions") / col("active_months"))\
    .withColumn("avg_rev_per_month", col("total_revenue") / col("active_months"))

In [0]:
avg_stats = dfq4_prep.select(avg("avg_tx_per_month").alias("global_avg_tx"), avg("avg_rev_per_month").alias("global_avg_rev")).collect()[0]

global_avg_tx = avg_stats["global_avg_tx"]
global_avg_rev = avg_stats["global_avg_rev"]

In [0]:
dfq4_popular_merchants = dfq4_prep.where((col("avg_tx_per_month") > global_avg_tx) & (col("avg_rev_per_month") > global_avg_rev))\
    .select("merchant_id")

In [0]:
dfq4_cities = df.where("authorized_flag = 'Y' AND calc_city_id != -1 AND merchant_id IS NOT NULL")\
    .selectExpr("merchant_id", "calc_city_id AS city_id")\
    .distinct()

In [0]:
dfq4_final = dfq4_popular_merchants.join(dfq4_cities, "merchant_id", "left")\
    .groupBy("city_id")\
    .agg(countDistinct("merchant_id").alias("no_of_popular_merchants"))\
    .orderBy(desc("no_of_popular_merchants"))

#### `answer 4.1`:
Most popualar cities are _69_, _1_, and _333_

In [0]:
display(dfq4_final.limit(10))

#### `answer 4.2`:
Looking at contingency table, there doesn't seem to be any obvious correlation between location and categories

In [0]:
df.where("authorized_flag = 'Y' AND merchant_name != 'Unknown' AND category != 'Unknown' AND calc_city_id != -1")\
    .crosstab('calc_city_id', 'category')\
    .show()

### question 5: 
A new merchant is coming in to do business and you have been assigned to give
advice based strictly on the historical transactions. You are expected to provide a response to the following questions.

#### question 5a: 
Which cities would you advise them to focus on and why?

In [0]:
dfq5a_cities = df.where("authorized_flag = 'Y' AND calc_city_id != -1")\
    .groupBy("calc_city_id")\
    .agg(sum("purchase_amount").alias("total_rev"), countDistinct("merchant_id").alias("merchant_count"))\
    .withColumn("revenue_per_merchant", round(col("total_rev") / col("merchant_count"),0))\
    .orderBy(desc("revenue_per_merchant"))

#### `answer 5a`: 
Looking at results, it seems that **city 69** and **city 1** have biggest revenue per merchant, so I would focus on these.

In [0]:
display(dfq5a_cities.limit(10))

#### question 5b: 
Which categories would you recommend they sell?

In [0]:
dfq5b_cat69 = df.where("authorized_flag = 'Y' AND category != 'Unknown' AND calc_city_id = 69")\
    .groupBy("category").agg(sum("purchase_amount").alias("total_rev"), avg("purchase_amount").alias("avg_purchase"), count("*").alias("transaction_count"))\
    .withColumn("total_rev", round("total_rev", 0))\
    .orderBy(desc("total_rev"))

In [0]:
dfq5b_cat1 = df.where("authorized_flag = 'Y' AND category != 'Unknown' AND calc_city_id = 1")\
    .groupBy("category").agg(sum("purchase_amount").alias("total_rev"), avg("purchase_amount").alias("avg_purchase"), count("*").alias("transaction_count"))\
    .withColumn("total_rev", round("total_rev", 0))\
    .orderBy(desc("total_rev"))

#### `answer 5b`: 
Looking at the results, its best to focus on category **A** if _city 69_ is selected, and category **B** if _city 1_ is selected, since they produce the biggest revenue and have the highest transaction frequency

In [0]:
display(dfq5b_cat69.limit(10))

In [0]:
display(dfq5b_cat1.limit(10))

#### question 5c: 
Are there particular periods (months) that have interesting sales behaviors?

In [0]:
dfq5c_year_months = df.where("authorized_flag = 'Y'")\
    .groupBy("purchase_year", "purchase_month") \
    .agg(sum("purchase_amount").alias("monthly_total"))

In [0]:
dfq5c_months = dfq5c_year_months.groupBy("purchase_month") \
    .agg(avg("monthly_total").alias("avg_monthly_sales"))\
    .withColumn("avg_monthly_sales", round("avg_monthly_sales", 0))\
    .orderBy("purchase_month")

#### `answer 5c`: 
Looking at the results, _December_ was a month with the biggest sales. This is expected since this is famously the biggest shopping month in the whole world. 
One interesting thing here is strong sales increase starting in _July / August_, which can't be explained easily. My expectation are that the sales would be lower in these months since people are traveling and spending less?

In [0]:
display(dfq5c_months)

#### question 5d: 
What hours would you recommend they open and close for the day?

In [0]:
dfq5d_hours = df.where("authorized_flag = 'Y'")\
    .select("purchase_date", "purchase_hour")\
    .withColumn("purchase_hour_clean", concat(lpad(col("purchase_hour").cast(StringType()),2,0), lit("00")))\
    .withColumn("is_weekend", dayofweek("purchase_date").isin([1,7]).cast("int"))

In [0]:
dfq5d_final = dfq5d_hours.groupBy("purchase_hour", "purchase_hour_clean", "is_weekend").agg(count("*").alias("no_of_transaction"))\
    .orderBy("is_weekend", "purchase_hour").drop("purchase_hour")

#### `answer 5d`: 
Looking at the results, for weekdays starting from 08:00 and up untill 23:00 we have lots of trafic, and for the weekends starting from 10:00 and then untill 20:00

In [0]:
display(dfq5d_final.limit(50))

#### question 5e: 
Would you recommend accepting payments in installments? Assume a credit default
rate of 22.9% per month.
For this question, consider the “installments” header in the historical transactions and the impact it may have, if any, on merchant sales (merchant sales in terms of
purchase_amounts). We are making a simplistic assumption that 25% of sales is gross
profit to merchants, there are equal installments and everyone who defaulted did so after making half payment.

In [0]:
dfq5e_inst = df.where("authorized_flag = 'Y' AND installments != 999")\
    .withColumn("has_installments", when(col("installments") >= 2, 1).otherwise(0))\
    .groupBy("has_installments")\
    .agg(avg("purchase_amount").alias("avg_invoice"), avg("installments").alias("avg_months") )

In [0]:
orders_cash = dfq5e_inst.collect() [0]["avg_invoice"]
orders_credit = dfq5e_inst.collect() [1]["avg_invoice"]
avg_months = dfq5e_inst.collect() [0]["avg_months"]

print(f"Average cash order: {orders_cash:.1f}")
print(f"Average installment order: {orders_credit:.1f}")

In [0]:
monthly_default_rate = 0.229
survival_rate = (1 - monthly_default_rate) ** avg_months
cumulative_default_risk = 1 - survival_rate

In [0]:
profit_margin = orders_cash * 0.25

exp_profit_credit = (orders_credit * 0.25 * survival_rate) + (orders_credit * -0.25 * cumulative_default_risk)

In [0]:
print(f"Avg installment duration: {avg_months:.1f} months")
print(f"Expected profit (Cash): {profit_margin:.1f}")
print(f"Expected profit (Installments): {exp_profit_credit:.1f}")

#### `answer 5e`: 
Looking at the results, we should NOT accept installments, and its because we are expected to take a profit loss on average, and also there doesn't seem to be any noticeable improvments in sales when installemnts are used as option: _average cash order: 20105_ vs _average installment order: 20100_